<a href="https://colab.research.google.com/github/sandeepkumar101010/DLCVNLP/blob/main/cat_dog_classification_final%20WITH%20VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Input Size of image 
IMAGE_SIZE = [224, 224]

#training and valid path 
train_path = r'/content/drive/MyDrive/Dog_cat with Transfer learning/train'
valid_path = r'/content/drive/MyDrive/Dog_cat with Transfer learning/test' 

In [4]:
#pulling out trained weights from VGG16
#include top = False as we will not use FC and O/p of VGG16 we will use till last max pooling 
#weights = imagenet name to get trained weights
#input_shape = Shape input for model
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [5]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False


In [10]:
#to get the number of output classes
folders = glob('/content/drive/MyDrive/Dog_cat with Transfer learning\*')

In [11]:
folders

[]

In [ ]:
glob(r'D:\anacondaenv\training_set\*')

1

In [6]:
x = Flatten()(vgg.output)
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_directory(r'/content/drive/MyDrive/Dog_cat with Transfer learning/train',
                                                 target_size=(224,224),
                                                 batch_size=32,
                                                 class_mode="categorical")

Found 18997 images belonging to 2 classes.


In [13]:
#Scaling test data before getting prediction from trained model
test_set = test_datagen.flow_from_directory(r'/content/drive/MyDrive/Dog_cat with Transfer learning/test',
                                            target_size=(224,224),
                                            batch_size = 32,
                                            class_mode="categorical")

Found 6003 images belonging to 2 classes.


In [14]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=30,
  validation_steps=50,
   )


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
30/30 [==============================] - 573s 19s/step - loss: 0.8664 - accuracy: 0.6521 - val_loss: 0.4533 - val_accuracy: 0.8119
Epoch 2/20
30/30 [==============================] - 486s 16s/step - loss: 0.4206 - accuracy: 0.8128 - val_loss: 0.2415 - val_accuracy: 0.9013
Epoch 3/20
30/30 [==============================] - 408s 14s/step - loss: 0.2462 - accuracy: 0.8907 - val_loss: 0.2418 - val_accuracy: 0.8950
Epoch 4/20
30/30 [==============================] - 367s 12s/step - loss: 0.2251 - accuracy: 0.9095 - val_loss: 0.2107 - val_accuracy: 0.9094
Epoch 5/20
30/30 [==============================] - 295s 10s/step - loss: 0.2222 - accuracy: 0.9086 - val_loss: 0.2471 - val_accuracy: 0.9000
Epoch 6/20
30/30 [==============================] - 269s 9s/step - loss: 0.3259 - accuracy: 0.8550 - val_loss: 0.4011 - val_accuracy: 0.8344
Epoch 7/20
30/30 [==============================] - 252s 9s/step - loss: 0.2399 - accuracy: 0.9028 - val_loss: 0.2295 - val_accuracy: 0.9038
Epoch 8/

In [ ]:
#plotting training and validation loss 
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label = 'train_loss')
plt.plot(r.history['val_loss'], label = 'val_loss')
ply.legend()
plt.show()
plt.savefig('LossVal_loss')


In [ ]:
#plotting training and validation accuracy
import matplotlib.pyplot as plt
plt.plot(r.history['accuracy'], label = 'train_accuracy')
plt.plot(r.history['val_accuracy'], label = 'val_accuracy')
ply.legend()
plt.show()
plt.savefig('train_valid_accuracy')

In [ ]:

# save it as a h5 file


from tensorflow.keras.models import load_model

cnn.save('model_rcat_transfer.h5')

In [15]:
y_pred = model.predict(test_set)
y_pred

array([[9.5694101e-01, 4.3058988e-02],
       [9.9887830e-01, 1.1216428e-03],
       [4.4236522e-02, 9.5576346e-01],
       ...,
       [4.4169663e-03, 9.9558300e-01],
       [2.6241744e-01, 7.3758262e-01],
       [3.1996701e-15, 1.0000000e+00]], dtype=float32)

In [20]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/Dog_cat with Transfer learning/test/Dog/dog.1000.jpg', target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [21]:
#as we have sue softmax activation function so it will give probablity of every category so there wiil bee two 
#output like[0.9953,0.0047]
a = np.argmax(result)

In [22]:
#so a will contain values on two indices a[0],a[1]
if a == [0]:
    print("The image classified is Cat")
else:
    print("The image classified is dog")

The image classified is dog
